In [1]:
from ipyparallel import Client
import os
import pprint
pp = pprint.PrettyPrinter(indent=4)
rc = Client()
data_path = os.path.abspath("data")
res_path = os.path.abspath("results")
random_path = os.path.abspath("random")
dview = rc.load_balanced_view()
#dview = rc.direct_view('all')
print 'n of workers:', len(dview)
print len(rc)
pp.pprint(dview.queue_status())

n of workers: 54
54
{   0: {   u'completed': 4106, u'queue': 0, u'tasks': 0},
    1: {   u'completed': 4256, u'queue': 0, u'tasks': 0},
    2: {   u'completed': 4195, u'queue': 0, u'tasks': 0},
    3: {   u'completed': 4228, u'queue': 0, u'tasks': 0},
    4: {   u'completed': 4104, u'queue': 0, u'tasks': 0},
    5: {   u'completed': 4067, u'queue': 0, u'tasks': 0},
    6: {   u'completed': 4089, u'queue': 0, u'tasks': 0},
    7: {   u'completed': 4093, u'queue': 0, u'tasks': 0},
    8: {   u'completed': 4010, u'queue': 0, u'tasks': 0},
    9: {   u'completed': 4059, u'queue': 0, u'tasks': 0},
    10: {   u'completed': 4043, u'queue': 0, u'tasks': 0},
    11: {   u'completed': 4204, u'queue': 0, u'tasks': 0},
    12: {   u'completed': 4083, u'queue': 0, u'tasks': 0},
    14: {   u'completed': 4053, u'queue': 0, u'tasks': 0},
    15: {   u'completed': 3982, u'queue': 0, u'tasks': 0},
    16: {   u'completed': 3988, u'queue': 0, u'tasks': 0},
    17: {   u'completed': 3954, u'queue': 0, u

In [2]:
%%px --local
import os
import pandas as pd
from string import strip
import numpy as np
from string import strip
from itertools import combinations
from scipy import stats
import time



In [9]:
values_df = pd.DataFrame.from_csv(os.path.join(data_path,'in_df.txt'),sep='\t')
cluster_res = pd.DataFrame.from_csv('res_clustering.csv')
print values_df.iloc[:3,:3]
print cluster_res.iloc[:3,:3]
pairs = []
for index, cut_distance in enumerate(cluster_res.columns):
    temp_df = pd.concat([values_df,cluster_res[cut_distance]],1)
    n_of_clusters = cluster_res[cut_distance].value_counts()
    n_of_clusters = n_of_clusters[n_of_clusters>=2]
    for cluster_id in n_of_clusters.index.values:
        genes = temp_df[temp_df[cut_distance]==cluster_id].index.values
        temp_pairs = list(combinations(genes,2))
        temp_pairs = [(n[0],n[1],cut_distance,cluster_id) for n in temp_pairs]
        pairs+=temp_pairs
print len(pairs)


                  median_1     median_2     median_3
Tb927.10.3210  688310000.0  276010000.0  215760000.0
Tb927.5.2310           0.0          0.0          0.0
Tb927.8.7570           0.0          0.0          0.0
               clusters_0.0  clusters_0.06  clusters_0.09
Tb927.10.3210          2325           2324           2323
Tb927.5.2310           1801           1800           1800
Tb927.8.7570           3018           3017           3016
29319788


<AsyncResult: _push>

In [4]:
split = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]
batches = [n for n in split(pairs, 10000)]
print len(batches)

2932


In [10]:
rc[:].push(dict(
    values_df=values_df       
))

<AsyncResult: _push>

In [11]:
%%px --local
def process(batch):
    print 'start'
    res = [ [str(stats.pearsonr(values_df.loc[n[0]], values_df.loc[n[1]])[0]),str(n[2]),str(n[3])] for n in batch]
    return res

In [14]:

start_time = time.time()
tasks = []
for batch in batches:
    ar = dview.apply_async(process, batch)
    tasks.append(ar)
print '-------send to-------------\n'     
print time.time() - start_time
print 'min: ', (time.time() - start_time)/60
print 'h: ', (time.time() - start_time)/60/60
print '-------------------------\n'


-------send to-------------

143.868999958
min:  2.39781666597
h:  0.0399636110995
-------------------------



In [15]:
pp.pprint(dview.queue_status())
print len(tasks)
#print tasks[0].get()[0][0]

{   0: {   u'completed': 4203, u'queue': 0, u'tasks': 1},
    1: {   u'completed': 4354, u'queue': 0, u'tasks': 1},
    2: {   u'completed': 4293, u'queue': 0, u'tasks': 0},
    3: {   u'completed': 4326, u'queue': 0, u'tasks': 1},
    4: {   u'completed': 4201, u'queue': 0, u'tasks': 1},
    5: {   u'completed': 4164, u'queue': 0, u'tasks': 1},
    6: {   u'completed': 4186, u'queue': 0, u'tasks': 0},
    7: {   u'completed': 4190, u'queue': 0, u'tasks': 1},
    8: {   u'completed': 4107, u'queue': 0, u'tasks': 1},
    9: {   u'completed': 4156, u'queue': 0, u'tasks': 1},
    10: {   u'completed': 4139, u'queue': 0, u'tasks': 1},
    11: {   u'completed': 4302, u'queue': 0, u'tasks': 1},
    12: {   u'completed': 4179, u'queue': 0, u'tasks': 1},
    14: {   u'completed': 4150, u'queue': 0, u'tasks': 1},
    15: {   u'completed': 4078, u'queue': 0, u'tasks': 1},
    16: {   u'completed': 4085, u'queue': 0, u'tasks': 1},
    17: {   u'completed': 4051, u'queue': 0, u'tasks': 1},
    18:

In [16]:

tasks = [n.get() for  n in tasks]
file_res  = open('person_res.txt','w')
for batch in tasks:
    for item in batch:
       file_res.write('\t'.join(item)+'\n') 
file_res.close()
